# BAU Proteinas / Imperdonables/ Low Penetration

In [61]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification

In [62]:
city_code = "BOG"

In [63]:
#->>> ACA TENGO DUDA, PORQUE EN EL DE NB SE COLOCA 3 MESES, 1 MES Y 2 MESES Y ACA ASI?
def definir_fechas(city_code):
    
    def format_date(date):
        return date.strftime("%Y-%m-%d")

    if  city_code       == "BAQ":
        dias_ventas     = 70
        dias_non_buyers = 23
    
    elif city_code == "BOG":
        dias_ventas = 70
        dias_non_buyers = 23
        
    elif city_code == "MDE":
        dias_ventas = 70
        dias_non_buyers = 23
    
    elif city_code == "CMX":
        dias_ventas = 70
        dias_non_buyers = 23
    
    elif city_code == "GDL":
        dias_ventas = 70
        dias_non_buyers = 23
    
    elif city_code == "SPO":
        dias_ventas = 70
        dias_non_buyers = 23
    
    elif city_code == "BHZ":
        dias_ventas = 70
        dias_non_buyers = 23   
        
    elif city_code == "CWB":
        dias_ventas = 70
        dias_non_buyers = 23 
        
    today_date = date.today()
    today = format_date(date=today_date)
    
    non_buyers_date = date.today()- timedelta(days=dias_non_buyers)
    non_buyers = format_date(date=non_buyers_date)
    
    ventas_date = date.today()- timedelta(days=dias_ventas)
    ventas = format_date(date=ventas_date)
    
    return today, non_buyers, ventas

business_type_id = 1
exchange_rate_dict = {
    "BOG":3000,
    "BAQ":3000,
    "MDE":3000,
    "CMX":19.19,
    "SPO":3.88,
    "BHZ":3.88,
    "GDL":19.19,
    "CWB":3.88
}
#MIRAR ESTEEEEEEEEEEE
#//MicroSegmented users https://redash.federate.frubana.com/queries/82561
users_eliminar_ms = pd.read_csv("import_data/segments/Segments_W16.csv")["customer_id"].unique().tolist()

#//KAMs customers ### Revisar la columna como esta escrito el customer_id me lo manda Emiliano Lomeli -> KAMs
users_eliminar_kam = pd.read_csv("import_data/segments/KAMs_APR.csv", sep=',')["user_id"].unique().tolist()
#//Old users queda guardado como BAU_{city_code}_ids_semana.csv

users_eliminar_prev = pd.read_csv(f'import_data/BAU clientes/BAU_{city_code}_ids_semana_pasada_3.csv')["customer_id"].unique().tolist()
#users_eliminar_prev2 = pd.read_csv(f'import_data/BAU clientes/BAU_{city_code}_ids_semana_pasada_2.csv')["customer_id"].unique().tolist()

users_eliminar = users_eliminar_ms + users_eliminar_kam + users_eliminar_prev #+ users_eliminar_prev2
#+ users_eliminar_prev

In [26]:
print(len(users_eliminar))
print(len(users_eliminar_kam))
#print(len(users_eliminar_prev))
#print(len(users_eliminar_prev2))

95673
1737


In [27]:
################################################
###Cuello de botella aca
################################################
date_today, date_non_buyers, date_ventas = definir_fechas(city_code=city_code)
exchange_rate = exchange_rate_dict[city_code]
ventas_todos = data_ventas_query(ciudad=city_code, tipo_cambio=exchange_rate,
                           desde=date_ventas, hasta=date_today, tipo_negocio=business_type_id)

In [28]:
ventas_todos['category'] = ventas_todos['category'].apply(lambda x: 'Bebidas' if 'Cervezas y Licores' in x else x)

In [29]:
#//Remove skus ###hablar con Paula Diaz --equipo de Ara 
skus_remove = [
271395,
271396,
271392,
99017,
324145,
324143,
56606,
56612,
-301818, #Diana
124661, #Diana
124659, #Diana
56612, #alquería
401540, #alquería 
56606, #alquería
90176, #alquería
266755, #alquería
126505, #alquería
#### experimento mayo
-303030,
124708,
124710,
-303057,
124702,
124700,
124716,
63697,
280865,
280868,
280862,
440367,
-304254,
-302913,
277542,
-300543,
62912,
212438,
-302130,
210161,
210159,
97137,
83969,
241989,
271594,
271602,
99017,
147339,
254113,
254112,
31507,
82996,
102355,
111139,
118025,
118027,
118020,
-304413,
48733,
-302271,
-302286,
367833,
136080,
136082,
130690,
290511,
169189,
290224,
290223,
290222,
377870,
377869,
377868,
219702,
37211,
219697,
37188,
296040,
296041,
91360,
91356,
106890,
106889,
106895,
162844,
162845,
162843,
278268,
322342,
278267,
285339,
285338,
195864,
195961,
195960,
290048,
290049,
290047,
218839,
218840,
218841,
197104,
197105,
359613,
196789,
196790,
196791,
250239,
250240,
197116,
342968,
342969,
342967,
197114,
197113,
197718,
197719,
197720,
195993,
195992,
195994,
304504,
249435,
249433,
249434,
290283,
371569,
371567,
371571,
371573,
254974,
254842,
254971,
254844,
254865,
254856,
295085,
254973,
336632,
249954,
249953,
249993,
249992,
404804,
404799
]

In [30]:
ventas_todos = ventas_todos[~ventas_todos['sku_id'].isin(skus_remove)]

In [31]:
ventas = ventas_todos[(~ventas_todos["customer_id"].isin(users_eliminar))].copy()

In [32]:
ventas_todos.customer_id.nunique()

20405

In [33]:
ventas.customer_id.nunique()

5604

In [34]:
ventas.customer_id.to_csv(f"import_data/BAU clientes/BAU_{city_code}_ids_semana_pasada_2.csv", index=False)

In [35]:
ventas.category.unique()

array(['Proteínas', 'Abarrotes', 'Frutas & Verduras', 'Congelados',
       'Aseo e Higiene', 'Bebidas', 'Desechables', 'Lácteos & Huevos'],
      dtype=object)

In [36]:
ventas.subcat.unique()

array(['Cerdo', 'Salsas', 'Pollo', 'Frutas', 'Verduras', 'Res',
       'Alimentos Congelados', 'Pasta', 'Pescados & Mariscos Congelado',
       'Aceites & Grasas', 'Implementos de Aseo', 'Bebidas sin alcohol',
       'Contenedores', 'Tubérculos', 'Arroz', 'Azúcar & Endulzantes',
       'Cubiertos & Pitillos', 'Papel & Toallas',
       'Detergente, Jabón & Lavalozas', 'Granos', 'Vasos', 'Leches',
       'Sal & Sazonadores', 'Bolsas', 'Frutas Jugo', 'Quesos',
       'Desinfectantes & Sanitizantes', 'Empaque & Envoltura',
       'Embutidos', 'Pulpas de Fruta', 'Derivados Lácteos',
       'Platos & Bandejas', 'Huevos', 'Harinas & Mezclas', 'Mantequillas',
       'Cremas', 'Complementos', 'Café, Chocolate e Infusiones',
       'Especias', 'Enlatados', 'Panadería & Repostería', 'Dotación',
       'Cervezas & Licores'], dtype=object)

In [65]:
ventas[['category','subcat']].loc[ventas.category.isin(['Proteínas'])].subcat.unique()

array(['Cerdo', 'Pollo', 'Res', 'Pescados & Mariscos Congelado',
       'Embutidos'], dtype=object)

## Proteinas

In [38]:
defined_categories = {}
defined_categories['BOG'] = ['Pollo', 'Res', 'Cerdo','Pescados & Mariscos Congelado','Embutidos']
defined_categories['MDE'] = []
defined_categories['BAQ'] = ['Pollo', 'Res', 'Cerdo','Pescados & Mariscos Congelado','Embutidos']
defined_categories['CMX'] = ['Pollo', 'Res', 'Cerdo','Pescados & Mariscos Congelado','Embutidos']
defined_categories['GDL'] = ['Pollo', 'Res', 'Cerdo','Pescados & Mariscos','Embutidos']
defined_categories['SPO'] = ['Frango', 'Suínos', 'Embutidos', 'Carne Vermelha','Peixes e Frutos do Mar']
defined_categories['CWB'] = ['Frango', 'Suínos', 'Carne Vermelha','Peixes e Frutos do Mar']
defined_categories['BHZ'] = ['Frango', 'Suínos', 'Embutidos', 'Carne Vermelha','Peixes e Frutos do Mar']

In [39]:
csv_prot, nb_id_prot, nb_email = proceso_para_csv(data_ventas=ventas, todays_date=date_today, non_buyers_date=date_non_buyers, tipo ='non_buyers'
                                        , city=city_code, list_to_filter=defined_categories[city_code], level='subcat', limite=0.70, data_ventas_skus = ventas_todos)
#->>> ACAA COLOCAN LAS FECHAS DE LAS CAMPAÑAS CON proceso_para_csv
#tipo is very important

Uso de data externa


In [40]:
csv_prot.to_csv(f"export_data/Business_As_Usual/{city_code}/proteinas_{city_code}.csv", index=False)
nb_id_prot.to_csv(f"export_data/Business_As_Usual/{city_code}/segmentos_proteinas_{city_code}.csv", index=False)

## Imperdonables

In [41]:
defined_categories_dict = {
    "BOG":["Tubérculos", "Verduras", "Frutas", "Huevos", "Aceites & Grasas", "Vasos", "Contenedores", "Papel & Toallas"
           ,"Arroz",'Frutas Jugo','Bebidas sin alcohol'], 
    "MDE":["Tubérculos", "Verduras", "Frutas", "Huevos", "Vasos", "Contenedores", "Papel & Toallas"
           ,"Arroz",'Bebidas Sin Alcohol'], 
    "BAQ":["Tubérculos", "Arroz", "Frutas", "Huevos", "Verduras", "Vasos", "Contenedores","Detergente, Jabón & Lavalozas",'Bebidas sin alcohol'],
    "CMX":["Azúcar & Endulzantes", "Tubérculos", "Granos", "Frutas", "Verduras", "Huevos","Leches",
           "Vasos, Contenedores & Platos","Detergente, Jabón & Lavatrastes","Arroz",'Pasta'], 
    "GDL":["Enlatados", "Tubérculos", "Verduras", "Frutas",'Alimentos Congelados', 'Leches', 'Huevos',
           "Vasos, Contenedores & Platos","Detergente, Jabón & Lavatrastes", "Papel & Toallas",'Refrescos'], 
    "SPO":["Tubérculos", "Verduras", "Frutas", "Legumes",'' "Leite","Ovos","Detergente, Sabão e Lava-Louças","Feijão"
          ,'Farinhas e Misturas','Cerveja', 'Refrigerante', 'Suco', 'Água'],
    "CWB":["Tubérculos", "Verduras", "Frutas", "Legumes",'' "Leite","Ovos","Detergente, Sabão e Lava-Louças","Feijão"
           ,"Arroz",'Farinhas e Misturas','Cerveja', 'Refrigerante', 'Suco', 'Água'],
    "BHZ":["Tubérculos", "Verduras", "Frutas", "Legumes",'' "Leite","Ovos","Detergente, Sabão e Lava-Louças","Feijão"
           ,"Arroz",'Farinhas e Misturas','Cerveja', 'Refrigerante', 'Suco', 'Água'] 
}



defined_categories = defined_categories_dict[city_code]

csv_imper, nb_id_imper, nb_email = proceso_para_csv(data_ventas=ventas, todays_date=date_today, non_buyers_date=date_non_buyers, 
                                        city=city_code, list_to_filter=defined_categories, level='subcat', limite=0.70, data_ventas_skus = ventas_todos)

# # resumen_ara.to_csv(f"data/Jul-/resumen_ara_imperdonables_{city_code}.csv", index=False)
csv_imper.to_csv(f"export_data/Business_As_Usual/{city_code}/imperdonables_{city_code}.csv", index=False)
nb_id_imper.to_csv(f"export_data/Business_As_Usual/{city_code}/segmentos_imperdonables_{city_code}.csv", index=False)
print(f'Done {city_code}')
assert 1 == 1

Uso de data externa
Done BOG


## Low Pen

In [42]:
defined_categories_dict = {
    "BOG":[ "Congelados", "Aseo e Higiene", "Lácteos & Huevos", "Desechables"],
    "MDE":[],
    "BAQ":[ "Congelados", "Lácteos & Huevos", "Aseo e Higiene", "Desechables"],
    "CMX":["Congelados", "Lácteos & Huevos", "Aseo e Higiene", "Desechables"],
    "GDL":["Congelados", "Lácteos & Huevos", "Desechables"],
    "SPO":[ "Congelados", 'Laticínios e Ovos','Limpeza e Higiene','Descartáveis'],
    "CWB":[ 'Laticínios e Ovos','Limpeza e Higiene','Descartáveis'],
    "BHZ":[ "Congelados", 'Laticínios e Ovos','Limpeza e Higiene','Descartáveis']
}

defined_categories = defined_categories_dict[city_code]

In [43]:
#//Fix para excluir subcategorias
subcat_ex = {'BOG':['Huevos','Vasos, contenedores & platos','Papel & toallas'],
             'BAQ':['Huevos','Vasos, contenedores & platos','Detergente, Jabón & Lavalozas'],
             'CMX':['Huevos','Leches','Vasos, Contenedores & Platos','Detergente, Jabón & Lavatrastes'],
             'GDL':['Huevos','Leches','Vasos, Contenedores & Platos'],
             'SPO':['Ovos','Leite','Detergente, Sabão e Lava-Louças'],
             'CWB':['Ovos','Leite','Detergente, Sabão e Lava-Louças'],
             'BHZ':['Ovos','Leite','Detergente, Sabão e Lava-Louças']}

In [44]:
def organize_text(x, city):
    def normalize_text(raw_text):
        """
        Removes common accent characters and normalize text.
        """
        raw_text = raw_text.replace(" ", "")
        raw_text = re.sub(r'\s*', '', raw_text)
        raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
        raw_text = re.sub(r"[èéêë]", 'e', raw_text)
        raw_text = re.sub(r"[ìíîï]", 'i', raw_text)
        raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
        raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
        raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
        raw_text = re.sub(r"[ß]", 'ss', raw_text)
        raw_text = re.sub(r"[ñ]", 'n', raw_text)
        raw_text = raw_text.lower()
        return raw_text
    today_date = date.today()
    y = today_date.strftime("%Y%m%d")
    x = normalize_text(x)
    x = x.replace("&", "y")
    x = y + "_"+ city + "_NB_" + x  
    return x

In [46]:
top_products_df = analizar_top_productos(data_ventas=ventas_todos)

hijo_padre_dict = dict(zip(top_products_df["sku_id"],
                           top_products_df["padre_sku_id"])) #ESTA ES LA TARJETA!!

top_products_df["sku_id_string"] = top_products_df["sku_id"].astype(str)
top_products_df["padre_sku_id_string"] = top_products_df["padre_sku_id"].astype(str)
padre_hijo_dict = top_products_df.groupby(by=["padre_sku_id_string"])["sku_id_string"].apply('#'.join).to_dict() #UN DICCIONARIO CON ID_TARJETA:[ID_HIJO1#ID_HIJO2....]

def crear_lista_de_skus(x, dictionary_1, dictionary_2): #->>> ACAA 
    threshold_products = x["penetration_porcentual"].quantile(q=0.70) # Cambiar para +/- productos
    list_of_sku_ids = x[x["penetration_porcentual"] >= threshold_products]["sku_id"].tolist()
    list_of_sku_ids_padre = list(map(dictionary_1.get, list_of_sku_ids))
    list_of_sku_ids_padre = [str(x) for x in list_of_sku_ids_padre]
    list_of_sku_ids_hijos = list(map(dictionary_2.get, list_of_sku_ids_padre))
    return "#".join(list_of_sku_ids_hijos)

top_products_df = top_products_df.drop(labels=["sku_id_string", "padre_sku_id_string"], axis=1)

cat_subcat = {}
for i in defined_categories:
    cat_subcat[i] = list(ventas_todos[ventas_todos["category"] == i]["subcat"].unique())

dict_skus_by_non_buyers = dict(zip([x for x in cat_subcat.keys()],
                                   ["#".join(top_products_df.groupby(by=["subcat"]).apply(crear_lista_de_skus, hijo_padre_dict, padre_hijo_dict)[cat_subcat[x]].values) for x in cat_subcat.keys()]))

csv_file = {}
start_date = (datetime.strptime(date_today, "%Y-%m-%d") + timedelta(days=0)).strftime("%d-%m-%Y")
end_date = (datetime.strptime(date_today, "%Y-%m-%d") + timedelta(days=8)).strftime("%d-%m-%Y")

csv_file["offer_name"] = [organize_text(x, city=city_code) for x in list(dict_skus_by_non_buyers.keys())]
csv_file["offer_description"] = None
csv_file["discount"] = 18
csv_file["automatically_consider_offer"] = "true"
csv_file["start_date"] = start_date
csv_file["end_date"] = end_date
csv_file["max_uses_per_order"] = 12
csv_file["max_uses_per_customer"] = 1
csv_file["customer_segment_id"] = None
csv_file["sku_id"] = list(dict_skus_by_non_buyers.values())
csv_lowpen = pd.DataFrame(csv_file)

todos_los_sku = []
for i in dict_skus_by_non_buyers.keys():
    lista_sku_ids = dict_skus_by_non_buyers[i]
    lista_sku_ids = lista_sku_ids.split("#")
    lista_sku_ids = [int(x) for x in lista_sku_ids]
    todos_los_sku.extend(lista_sku_ids)
    
resumen_ara = analizar_top_productos(data_ventas=ventas)
resumen_ara = resumen_ara[resumen_ara["sku_id"].isin(todos_los_sku)]

# resumen_ara.to_csv(f"data/Agosto/Business_As_Usual/{city_code}/resumen_ara_low_pen_{city_code}.csv", index=False)
csv_lowpen.to_csv(f"export_data/Business_As_Usual/{city_code}/low_pen_{city_code}.csv", index=False)

params_non_buyers = {}
for n, (i, j) in enumerate(zip([organize_text(x, city=city_code) for x in defined_categories], defined_categories)):
    params_non_buyers[n] = {
        "nombre":i,
        "featureFiltrado":"category",
        "valueFiltrado":[j]
    }

nb_id_lowpen = sacar_non_buyers(data_ventas=ventas, filtros=params_non_buyers, especificar_rango={"desde":date_non_buyers, "hasta":date_today})
customer_id_and_email = dict_cliente_email_from_ventas(data_ventas=ventas)
#non_buyers_email = non_buyers_df.stack().map(customer_id_and_email).unstack()
nb_id_lowpen.to_csv(f"export_data/Business_As_Usual/{city_code}/segmentos_low_pen_{city_code}.csv", index=False)

## Joint data

In [26]:
joint_csv = csv_imper.append(csv_prot).append(csv_lowpen)
joint_csv.to_csv(f"export_data/Business_As_Usual/{city_code}/BAU_{city_code}.csv", index=False)

In [27]:
joint_NB = pd.concat([nb_id_imper, nb_id_prot, nb_id_lowpen], ignore_index=True, sort=False)
print(joint_NB.nunique())

20220517_BOG_NB_tuberculos_W20                    4361
20220517_BOG_NB_verduras_W20                      2379
20220517_BOG_NB_frutas_W20                        3754
20220517_BOG_NB_huevos_W20                        5157
20220517_BOG_NB_aceitesygrasas_W20                3817
20220517_BOG_NB_vasos_W20                         4305
20220517_BOG_NB_contenedores_W20                  4162
20220517_BOG_NB_papelytoallas_W20                 4228
20220517_BOG_NB_arroz_W20                         4186
20220517_BOG_NB_frutasjugo_W20                    5077
20220517_BOG_NB_bebidassinalcohol_W20             4294
20220517_BOG_NB_pollo_W20                         4248
20220517_BOG_NB_res_W20                           4733
20220517_BOG_NB_cerdo_W20                         4429
20220517_BOG_NB_pescadosymariscoscongelado_W20    5351
20220517_BOG_NB_embutidos_W20                     5225
20220517_BOG_NB_congelados                        4640
20220517_BOG_NB_aseoehigiene                      2939
20220517_B

In [79]:
lst = []
name = f"export_data/Business_As_Usual/{city_code}/segmentos_BAU_{city_code}.txt"
for seg in joint_NB.columns:
    id_list = joint_NB[seg].dropna().astype(int)
    d = {'name':seg, 'customersIds': list(id_list)}
    lst.append(d)
with open(name, 'w') as outfile:
    json.dump(lst, outfile)    
print(f'Done {city_code}')

Done CWB
